<a href="https://colab.research.google.com/github/pkmariya/Scaler01/blob/master/ZeeRecommender_CaseStudy_Mariya_DSML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
The Objective is to create a Recommender System to show personalized movie recommendations based on ratings given by a user and other users similar to them in order to improve user experience.

In [18]:
# Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import io

import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import files
uploaded = files.upload()

Saving zee-ratings.dat to zee-ratings.dat
Saving zee-movies.dat to zee-movies.dat
Saving zee-users.dat to zee-users.dat


In [8]:
for fn in uploaded.keys():
  print('The file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

The file "zee-ratings.dat" with length 24594166 bytes
The file "zee-movies.dat" with length 171332 bytes
The file "zee-users.dat" with length 134410 bytes


In [45]:
users = pd.read_csv('zee-users.dat', sep='::', usecols=['UserID', "Gender", 'Age', 'Occupation', 'Zip-code'])
users.to_csv ('zee-users.csv', index=None)

In [51]:
movies = pd.read_csv('zee-movies.dat', sep='::', encoding='windows-1252', usecols=['Movie ID', 'Title', 'Genres'])
movies.to_csv('zee-movies.csv', index=None)
# movies_df = pd.read_csv(io.StringIO(uploaded['zee-movies.dat'].decode('utf-8')))

In [52]:
ratings = pd.read_csv('zee-ratings.dat', sep='::', usecols=['UserID', 'MovieID', 'Rating', 'Timestamp'])
ratings.to_csv ('zee-ratings.csv', index=None)

In [53]:
user_df = pd.read_csv('zee-users.csv')
movies_df = pd.read_csv('zee-movies.csv')
ratings_df = pd.read_csv('zee-ratings.csv')

In [54]:
user_df.describe(include='all')

,UserID,Gender,Age,Occupation,Zip-code
count,6040.000000,6040,6040.000000,6040.000000,6040
unique,NaN,2,NaN,NaN,3439
top,NaN,M,NaN,NaN,48104
freq,NaN,4331,NaN,NaN,19
mean,3020.500000,NaN,30.639238,8.146854,NaN
std,1743.742145,NaN,12.895962,6.329511,NaN
min,1.000000,NaN,1.000000,0.000000,NaN
25%,1510.750000,NaN,25.000000,3.000000,NaN
50%,3020.500000,NaN,25.000000,7.000000,NaN
75%,4530.250000,NaN,35.000000,14.000000,NaN


In [55]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   UserID      6040 non-null   int64 
 1   Gender      6040 non-null   object
 2   Age         6040 non-null   int64 
 3   Occupation  6040 non-null   int64 
 4   Zip-code    6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [56]:
user_df.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [73]:
user_df.nunique()

UserID        6040
Gender           2
Age              7
Occupation      21
Zip-code      3439
dtype: int64

In [57]:
movies_df.describe(include='all')

,Movie ID,Title,Genres
count,3883.000000,3883,3883
unique,NaN,3883,301
top,NaN,Toy Story (1995),Drama
freq,NaN,1,843
mean,1986.049446,NaN,NaN
std,1146.778349,NaN,NaN
min,1.000000,NaN,NaN
25%,982.500000,NaN,NaN
50%,2010.000000,NaN,NaN
75%,2980.500000,NaN,NaN


In [58]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Movie ID  3883 non-null   int64 
 1   Title     3883 non-null   object
 2   Genres    3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [59]:
movies_df.head()

,Movie ID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [71]:
movies_df.nunique()

Movie ID    3883
Title       3883
Genres       301
dtype: int64

In [60]:
ratings_df.describe(include='all')

,UserID,MovieID,Rating,Timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [61]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   UserID     1000209 non-null  int64
 1   MovieID    1000209 non-null  int64
 2   Rating     1000209 non-null  int64
 3   Timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [62]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [65]:
# Shape of the datasets
print("Users dataset contains {0} Rows and {1} Columns".format(user_df.shape[0], user_df.shape[1]))
print("Movies dataset contains {0} Rows and {1} Columns".format(movies_df.shape[0], movies_df.shape[1]))
print("Ratings dataset contains {0} Rows and {1} Columns".format(ratings_df.shape[0], ratings_df.shape[1]))

Users dataset contains 6040 Rows and 5 Columns
Movies dataset contains 3883 Rows and 3 Columns
Ratings dataset contains 1000209 Rows and 4 Columns


In [78]:
# Check if common users exist b/w user_df and ratings_df
common_users = ratings_df[~ratings_df['UserID'].isin(user_df)]

In [79]:
common_users.nunique()

UserID         6040
MovieID        3706
Rating            5
Timestamp    458455
dtype: int64

In [74]:
# Check if common movies exist b/w movies_df and ratings_df
common_movieID = ratings_df[~ratings_df['MovieID'].isin(movies_df)]

In [77]:
common_movieID.nunique()

UserID         6040
MovieID        3706
Rating            5
Timestamp    458455
dtype: int64

In [90]:
# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
movies_df.rename(columns={'Movie ID': 'MovieID'}, inplace=True)

In [97]:
# Merge movies_df and ratings_df
movie_rating_df = ratings_df.merge(movies_df, how='right', on='MovieID')

In [98]:
movie_rating_df.nunique()

UserID         6040
MovieID        3883
Rating            5
Timestamp    458455
Title          3883
Genres          301
dtype: int64

In [99]:
user_df.nunique()

UserID        6040
Gender           2
Age              7
Occupation      21
Zip-code      3439
dtype: int64

In [113]:
# Merge user_df and movie_rating_df
final_df = user_df.merge(movie_rating_df, how='right', on='UserID')

In [103]:
final_df.nunique()

UserID          6040
Gender             2
Age                7
Occupation        21
Zip-code        3439
MovieID         3883
Rating             5
Timestamp     458455
Title           3883
Genres           301
dtype: int64

In [105]:
# Description of final_df of all three datasets
final_df.describe(include='all')

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genres
count,1.000209e+06,1000209,1.000209e+06,1.000209e+06,1000209,1.000386e+06,1.000209e+06,1.000209e+06,1000386,1000386
unique,NaN,2,NaN,NaN,3439,NaN,NaN,NaN,3883,301
top,NaN,M,NaN,NaN,94110,NaN,NaN,NaN,American Beauty (1999),Comedy
freq,NaN,753769,NaN,NaN,3802,NaN,NaN,NaN,3428,116905
mean,3.024512e+03,NaN,2.973831e+01,8.036138e+00,NaN,1.865526e+03,3.581564e+00,9.722437e+08,NaN,NaN
std,1.728413e+03,NaN,1.175198e+01,6.531336e+00,NaN,1.096030e+03,1.117102e+00,1.215256e+07,NaN,NaN
min,1.000000e+00,NaN,1.000000e+00,0.000000e+00,NaN,1.000000e+00,1.000000e+00,9.567039e+08,NaN,NaN
25%,1.506000e+03,NaN,2.500000e+01,2.000000e+00,NaN,1.030000e+03,3.000000e+00,9.653026e+08,NaN,NaN
50%,3.070000e+03,NaN,2.500000e+01,7.000000e+00,NaN,1.835000e+03,4.000000e+00,9.730180e+08,NaN,NaN
75%,4.476000e+03,NaN,3.500000e+01,1.400000e+01,NaN,2.770000e+03,4.000000e+00,9.752209e+08,NaN,NaN


In [114]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000386 entries, 0 to 1000385
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   UserID      1000209 non-null  float64
 1   Gender      1000209 non-null  object 
 2   Age         1000209 non-null  float64
 3   Occupation  1000209 non-null  float64
 4   Zip-code    1000209 non-null  object 
 5   MovieID     1000386 non-null  int64  
 6   Rating      1000209 non-null  float64
 7   Timestamp   1000209 non-null  float64
 8   Title       1000386 non-null  object 
 9   Genres      1000386 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 84.0+ MB


In [107]:
final_df.head()

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genres
0,1.0,F,1.0,10.0,48067,1,5.0,978824268.0,Toy Story (1995),Animation|Children's|Comedy
1,6.0,F,50.0,9.0,55117,1,4.0,978237008.0,Toy Story (1995),Animation|Children's|Comedy
2,8.0,M,25.0,12.0,11413,1,4.0,978233496.0,Toy Story (1995),Animation|Children's|Comedy
3,9.0,M,25.0,17.0,61614,1,5.0,978225952.0,Toy Story (1995),Animation|Children's|Comedy
4,10.0,F,35.0,1.0,95370,1,5.0,978226474.0,Toy Story (1995),Animation|Children's|Comedy


In [115]:
print("Shape of final_df is: ", final_df.shape)

Shape of final_df is:  (1000386, 10)


In [108]:
# Check if missing values exist
final_df.isna().sum()

UserID        177
Gender        177
Age           177
Occupation    177
Zip-code      177
MovieID         0
Rating        177
Timestamp     177
Title           0
Genres          0
dtype: int64

In [117]:
# Remove Missing values
final_df = final_df[final_df['UserID'].notna()]

In [118]:
final_df.isna().sum()

UserID        0
Gender        0
Age           0
Occupation    0
Zip-code      0
MovieID       0
Rating        0
Timestamp     0
Title         0
Genres        0
dtype: int64

In [119]:
final_df.shape

(1000209, 10)